In [4]:
from video_dataset import VideoFrameDataset
import os
import matplotlib.pyplot as plt
from torchvision import transforms as T
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
val_transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        # T.Normalize(
        #     mean=(0.485, 0.456, 0.406),
        #     std=(0.229, 0.224, 0.225)
        # ),

    ])

In [3]:
import random
# random.seed(0)
root = os.path.join(os.getcwd(), 'zaloai/train/videos')  # Folder in which all videos lie in a specific structure
annotation_file = os.path.join(root.replace('videos', ''), 'annotations.txt')  # A row for each video sample as: (VIDEO_PATH START_FRAME END_FRAME CLASS_ID)

""" DEMO 1 WITHOUT IMAGE TRANSFORMS """
dataset = VideoFrameDataset(
    root_path=root,
    annotationfile_path=annotation_file,
    num_segments=16,
    frames_per_segment=1,
    imagefile_template='img_{:05d}.jpg',
    transform=val_transform,
    test_mode=False
)

sample = dataset[random.randint(0, 351)]  # take first sample of dataset 
frames = sample[0]   # list of PIL images
label = sample[1]    # integer label
path = sample[2]
print(frames.shape)
print(path)
print(label)


Dataset Warning: video /home/vietlq4/zaloaichallenge/zaloai/train/videos/84 has 4 frames but the dataloader is set up to load (num_segments=16)*(frames_per_segment=1)=16 frames. Dataloader will throw an error when trying to load this video.


Dataset Warning: video /home/vietlq4/zaloaichallenge/zaloai/train/videos/393 has 5 frames but the dataloader is set up to load (num_segments=16)*(frames_per_segment=1)=16 frames. Dataloader will throw an error when trying to load this video.


Dataset Warning: video /home/vietlq4/zaloaichallenge/zaloai/train/videos/1146 has 4 frames but the dataloader is set up to load (num_segments=16)*(frames_per_segment=1)=16 frames. Dataloader will throw an error when trying to load this video.


Dataset Warning: video /home/vietlq4/zaloaichallenge/zaloai/train/videos/1669 has 6 frames but the dataloader is set up to load (num_segments=16)*(frames_per_segment=1)=16 frames. Dataloader will throw an error when trying to load this video.


Dataset Warning: video

In [2]:
df = pd.read_csv('/home/vietlq4/zaloaichallenge/zaloai/train/label.csv')

In [3]:
df.head()

,fname,liveness_score
0,1.mp4,0
1,2.mp4,1
2,3.mp4,1
3,5.mp4,0
4,7.mp4,1


In [14]:
X_train, X_test, y_train, y_test = train_test_split(df['fname'], df['liveness_score'], test_size=0.2, random_state=42)

In [17]:
X_train[0]

'1.mp4'

In [19]:
X_train = [train.replace('.mp4','') for train in X_train]

In [20]:
X_test = [test.replace('.mp4', '') for test in X_test]

In [21]:
import numpy as np
import os
import pandas as pd

path = '/home/vietlq4/zaloaichallenge/zaloai/train/videos'
annotation_path = '/home/vietlq4/zaloaichallenge/zaloai/train/label.csv'
annotation_out_file = '/home/vietlq4/zaloaichallenge/zaloai/train/train_annotations.txt'

# video_filenames = os.listdir(path)
label_df = None
label2id = {}
if len(annotation_path) != 0:
    label_df = pd.read_csv(annotation_path)

    for idx, row in label_df.iterrows():
        label2id[row['fname'].replace('.mp4', '')] = int(row['liveness_score'])

with open(annotation_out_file, 'w') as f:

    for video_filename in sorted(X_train, key= lambda x: int(x)):
        start_frame = 0
        video_path = os.path.join(path, video_filename)
        try:
            num_frames = len(os.listdir(video_path))
            if num_frames == 0:
                print(f'{video_filename}- no frames')
                continue
        except FileNotFoundError as e:
            print(e)
            continue
            
        if len(annotation_path) == 0:
            annotation_string = "{} {} {}\n".format(
                video_filename, start_frame, num_frames - 1)
        else:
            annotation_string = "{} {} {} {}\n".format(
                video_filename, start_frame, num_frames - 1, label2id[video_filename])

        f.write(annotation_string)
